In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                          Dropout, Rescaling, RandomFlip, RandomRotation, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import zipfile
import os
import glob
import shutil

In [2]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


img_height,img_width = 224, 224
batch_size = 32
data_generator=ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = data_generator.flow_from_directory(
        'C:/Users/Asus/PycharmProjects/ML_Project/small_train',
        target_size=(img_width, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'C:/Users/Asus/PycharmProjects/ML_Project/small_val',
        target_size=(img_width, img_width),
        batch_size=batch_size,
        class_mode='categorical')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Asus/PycharmProjects/ML_Project/small_train'

In [30]:
base_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet', classes=2)
for layer in base_model.layers:
        layer.trainable=False

In [31]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [32]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l1(1e-4)))

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization_2 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 2)                 514       
                                                                 
Total params: 24,113,794
Trainable params: 525,570
Non-trainable params: 23,588,224
____________________________________

In [34]:
early_stop = EarlyStopping(monitor="val_loss",min_delta=0, patience=10,
                           verbose=0, mode="min", baseline=None, restore_best_weights=True)
check = ModelCheckpoint(filepath='convnet.keras', save_best_only=True, monitor='val_loss')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [35]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=SGD(learning_rate = 0.001, momentum = 0.1),
              metrics=['accuracy'])

In [ ]:
num_epoch = 10
model_log = model.fit_generator(train_generator,
                      epochs=num_epoch,
                      verbose=1,
                      validation_data=validation_generator,
                      callbacks = [early_stop, check, learning_rate_reduction])


model.save_weights('weights.h5')

C:\Windows\Temp\ipykernel_14144\731292204.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_log = model.fit_generator(train_generator,


Epoch 1/10
126/126 [==============================] - 281s 2s/step - loss: 0.3986 - accuracy: 0.8315 - val_loss: 0.2205 - val_accuracy: 0.9361 - lr: 0.0010
Epoch 2/10
126/126 [==============================] - 300s 2s/step - loss: 0.2439 - accuracy: 0.9117 - val_loss: 0.1991 - val_accuracy: 0.9437 - lr: 0.0010
Epoch 3/10
 73/126 [================>.............] - ETA: 1:53 - loss: 0.2032 - accuracy: 0.9332

In [ ]:
test_generator = data_generator.flow_from_directory(
    directory = 'C:/Users/Asus/PycharmProjects/ML_Project/test_dataset',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = None,
    shuffle = False,
    seed = 123
)
test_generator.reset()

pred = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)

predicted_class_indices = np.argmax(pred, axis = 1)
print(predicted_class_indices)

In [ ]:
from sklearn.metrics import accuracy_score
arr_zeros = np.zeros(1000)

# create an array of 600 ones
arr_ones = np.ones(600)

# concatenate the two arrays
result = np.concatenate((arr_zeros, arr_ones))
print(accuracy_score(result,predicted_class_indices))